In [1]:
# Перевірка підключення GPU
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [2]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.layers import Input, Dense, Layer, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall


from transformers import TFBertForSequenceClassification, TFBertModel

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split

**Підготовка кастомних класів і функцій**

Даний блок необхідно копіювати при завантажені моделей

In [3]:
# Кастомний шар для інтеграції з BERT
class BertLayer(Layer):
    def __init__(self, pretrained_model_name="bert-base-uncased", trainable=False, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        # Завантажуємо попередньо навчений BERT
        self.bert = TFBertModel.from_pretrained(pretrained_model_name)
        self.bert.trainable = trainable  # Заморожуємо або розморожуємо шари залежно від параметра trainable

    def call(self, inputs):
        # Вхідні дані: input_ids та attention_mask
        input_ids, attention_mask = inputs
        # Передаємо дані через BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state  # Повертаємо тільки last_hidden_state

In [4]:
'''
у якості метрики обрано Ф-1 у зв'язку із незбалансованістю класів. 
Підготуємо функцію для неї
'''

import tensorflow as tf
from tensorflow.keras import backend as K

def f1_metric(y_true, y_pred):
    # Преобразуем в бинарный формат для каждого класса
    y_true = K.cast(y_true, 'int32')
    y_pred = K.cast(K.greater_equal(y_pred, 0.5), 'int32')

    # Вычисляем точность (precision) и полноту (recall)
    true_positive = K.sum(K.cast(y_true * y_pred, 'float32'))
    false_positive = K.sum(K.cast((1 - y_true) * y_pred, 'float32'))
    false_negative = K.sum(K.cast(y_true * (1 - y_pred), 'float32'))

    precision = true_positive / (true_positive + false_positive + K.epsilon())
    recall = true_positive / (true_positive + false_negative + K.epsilon())

    # F1-score = 2 * (precision * recall) / (precision + recall)
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    
    return f1

In [5]:
'''
підготовка вагів для функції врат щоб врахувати незбалансованість вибірки
'''
# Підрахунок ваг для кожного класу
class_counts = {
    0: 15294,  # toxic
    1: 1595,   # severe_toxic
    2: 8449,   # obscene
    3: 478,    # threat
    4: 7877,   # insult
    5: 1405    # identity_hate
}

# Загальна кількість прикладів у вибірці
total_samples = 159571

# Вага для кожного класу буде пропорційною оберненому співвідношенню його частоти
class_weights = {}
for label, count in class_counts.items():
    # Вага класу розраховується як обернене відношення загальної кількості прикладів
    class_weights[label] = total_samples / count

# Нормалізація ваг класів, щоб їх сума була рівна 1
class_weights = {k: v / sum(class_weights.values()) for k, v in class_weights.items()}

In [6]:
import tensorflow as tf
from tensorflow.keras import backend as K

def weighted_f1_loss(y_true, y_pred, class_weights):
    """
    Кастомна функція втрат для оптимізації макро F1-міри з урахуванням ваг класів.
    
    Args:
        y_true: tf.Tensor, істинні мітки (розмірність [batch_size, num_classes]).
        y_pred: tf.Tensor, передбачення моделі (розмірність [batch_size, num_classes]).
        class_weights: dict, ваги класів (ключі - індекси класів, значення - ваги).

    Returns:
        tf.Tensor, значення функції втрат.
    """
    # Застосовуємо сигмоїду до передбачень, якщо вони ще не пройшли через активацію
    y_pred = K.sigmoid(y_pred)

    # Перетворення ваг класів у тензор
    class_weight_tensor = tf.constant([class_weights[i] for i in range(len(class_weights))], dtype=tf.float32)

    # Обчислення TP, FP, FN
    true_positives = K.sum(y_true * y_pred, axis=0)
    predicted_positives = K.sum(y_pred, axis=0)
    actual_positives = K.sum(y_true, axis=0)

    # Обчислення Precision та Recall для кожного класу
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (actual_positives + K.epsilon())

    # Обчислення F1 для кожного класу
    f1_per_class = 2 * (precision * recall) / (precision + recall + K.epsilon())

    # Застосування ваг класів
    weighted_f1 = f1_per_class * class_weight_tensor

    # Середнє значення макро F1
    macro_f1 = K.mean(weighted_f1)

    # Повернення від'ємного значення F1 як функції втрат (для мінімізації)
    return 1 - macro_f1

**Загальна підготовка даних**

In [7]:
data_path = '/kaggle/input/detaset/train_data.csv'
df = pd.read_csv(data_path)

# список категорій:
LABEL_COLUMNS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Конвертація токенізованих даних з рядків у масиви
for column in ['input_ids', 'attention_masks']:
    df[column] = df[column].apply(eval).apply(np.array)

# Виділяємо токенізовані вектори та мітки
input_ids = np.stack(df['input_ids'].values)
attention_mask = np.stack(df['attention_masks'].values)
labels = np.array(df[LABEL_COLUMNS].values)
labels = labels.astype('float32')

In [8]:
# Розділення на тренувальну та тестову вибірки (повний набір даних)

train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42
)

**Бінарна модель**

In [9]:
# Генерація міток для бінарної моделі
t_binary_labels = np.where(np.all(train_labels == 0, axis=1), 1, 0).astype('float32')
v_binary_labels = np.where(np.all(val_labels == 0, axis=1), 1, 0).astype('float32')

In [10]:
# Балансування даних

from imblearn.over_sampling import SMOTE

t_features = np.hstack((train_input_ids, train_attention_mask))

smote = SMOTE(random_state=42)
t_data_resampled, t_binary_labels_resampled = smote.fit_resample(t_features, t_binary_labels)

In [11]:
# До SMOTE
print("До обробки:")
print(f"Токсичні: {np.sum(t_binary_labels == 0)}")
print(f"Нетоксичні: {np.sum(t_binary_labels == 1)}")

# Після SMOTE
print("\nПісля обробки:")
print(f"Токсичні: {np.sum(t_binary_labels_resampled == 0)}")
print(f"Нетоксичні: {np.sum(t_binary_labels_resampled == 1)}")

До обробки:
Токсичні: 12981
Нетоксичні: 114675

Після обробки:
Токсичні: 114675
Нетоксичні: 114675


In [12]:
# Зворотне перетворення на train_input_ids і t_attention_mask

# Вихідні розміри train_input_ids и train_attention_mask
input_ids_size = train_input_ids.shape[1]
attention_mask_size = train_attention_mask.shape[1]

# Зворотній розподіл
t_input_ids_resampled = t_data_resampled[:, :input_ids_size]
t_attention_mask_resampled = t_data_resampled[:, input_ids_size:]

In [13]:
t_data = {
    "input_ids": t_input_ids_resampled,
    "attention_mask": t_attention_mask_resampled,
}

v_data = {
    "input_ids": val_input_ids,
    "attention_mask": val_attention_mask,
}

In [14]:
# Модель для бінарної класифікації

# Вхідні дані
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT шар
bert_outputs = BertLayer(trainable=False)([input_ids, attention_mask])

# Пулінг
pooled_output = GlobalAveragePooling1D()(bert_outputs)

# бінарна класифікація
binary_dense = Dense(128, activation="swish")(pooled_output)
binary_dropout = Dropout(0.3)(binary_dense)
binary_output = Dense(1, activation="sigmoid", name="binary_output")(binary_dropout)

# Модель
model_3_1 = Model(
    inputs=[input_ids, attention_mask],
    outputs=[binary_output]
)

# Компіляція моделі
model_3_1.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',        
    patience=3,                
    restore_best_weights=True  
)

# Навчання моделі
history_3_1 = model_3_1.fit(
    t_data,
    t_binary_labels_resampled,
    validation_data=(v_data, v_binary_labels),
    epochs=5,  
    batch_size=256,
    callbacks=[early_stopping]  
)

Epoch 1/5
896/896 ━━━━━━━━━━━━━━━━━━━━ 2011s 2s/step - accuracy: 0.5405 - loss: 0.6923 - val_accuracy: 0.5193 - val_loss: 0.7024
Epoch 2/5
896/896 ━━━━━━━━━━━━━━━━━━━━ 1979s 2s/step - accuracy: 0.5582 - loss: 0.6841 - val_accuracy: 0.4711 - val_loss: 0.7021
Epoch 3/5
896/896 ━━━━━━━━━━━━━━━━━━━━ 1980s 2s/step - accuracy: 0.5641 - loss: 0.6819 - val_accuracy: 0.5953 - val_loss: 0.6460
Epoch 4/5
896/896 ━━━━━━━━━━━━━━━━━━━━ 1981s 2s/step - accuracy: 0.5665 - loss: 0.6809 - val_accuracy: 0.5964 - val_loss: 0.6540
Epoch 5/5
896/896 ━━━━━━━━━━━━━━━━━━━━ 1981s 2s/step - accuracy: 0.5677 - loss: 0.6798 - val_accuracy: 0.6106 - val_loss: 0.6288


In [16]:
import json

# Збереження історії навчання в JSON
with open('history_3_1.json', 'w') as json_file:
    json.dump(history_3_1.history, json_file)


In [17]:
model_3_1.save("model_3_1.h5")

In [18]:
# Підготовка частини валідаціної вибірки для прогнозів 
_, test_input_ids, _, test_attention_mask, _, test_labels = train_test_split(
    val_input_ids, val_attention_mask, val_labels, test_size=0.1, random_state=42
)

test_binary_labels = np.where(np.all(test_labels == 0, axis=1), 1, 0).astype('float32')

In [19]:
# Отримання прогнозів для першої моделі
binary_predictions = model_3_1.predict(
    {'input_ids': test_input_ids, 'attention_mask': test_attention_mask},
    batch_size=64
)

binary_predictions = (binary_predictions > 0.5).astype(int)  # Перетворення в 0 або 1

# Розподіл прогнозів бінарної моделі
unique, counts = np.unique(binary_predictions, return_counts=True)
binary_distribution = dict(zip(unique, counts))

print("Розподіл міток:")
print(f"Нетоксичні (1): {binary_distribution.get(1, 0)}")
print(f"Токсичні (0): {binary_distribution.get(0, 0)}")

50/50 ━━━━━━━━━━━━━━━━━━━━ 42s 716ms/step
Розподіл міток:
Нетоксичні (1): 1956
Токсичні (0): 1236


In [20]:
# Розподіл істинних міток
unique, counts = np.unique(test_binary_labels, return_counts=True)
true_distribution = dict(zip(unique, counts))

print("Розподіл істинних міток:")
print(f"Нетоксичні (1): {true_distribution.get(1, 0)}")
print(f"Токсичні (0): {true_distribution.get(0, 0)}")

Розподіл істинних міток:
Нетоксичні (1): 2879
Токсичні (0): 313


In [21]:
# Оцінка моделі
print("\nКласифікаційний звіт для бінарної моделі:\n")
print(classification_report(test_binary_labels, binary_predictions, target_names=[
    "non_toxic", "toxic"
]))


Класифікаційний звіт для бінарної моделі:

              precision    recall  f1-score   support

   non_toxic       0.13      0.51      0.21       313
       toxic       0.92      0.63      0.75      2879

    accuracy                           0.62      3192
   macro avg       0.53      0.57      0.48      3192
weighted avg       0.84      0.62      0.69      3192



**Фінтюнінг бінарної моделі**

In [22]:
# Розморозка останніх 4-х шарів BERT
for layer in model_3_1.layers:
    if isinstance(layer, BertLayer):  
        for bert_layer in layer.bert.bert.encoder.layer[-4:]:  
            bert_layer.trainable = True

model_3_1.compile(
    optimizer=Adam(learning_rate=1e-5),  # Низкий learning rate для фінтюнинга
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [23]:
# Финтюнинг
history_3_1 = model_3_1.fit(
    t_data,
    t_binary_labels_resampled,
    validation_data=(v_data, v_binary_labels),
    epochs=3,               # Невелика кількість епох, оскільки БЕРТ має навчатись дуже швидко
    batch_size=256
)

Epoch 1/3
896/896 ━━━━━━━━━━━━━━━━━━━━ 2016s 2s/step - accuracy: 0.5727 - loss: 0.6776 - val_accuracy: 0.5466 - val_loss: 0.6649
Epoch 2/3
896/896 ━━━━━━━━━━━━━━━━━━━━ 1980s 2s/step - accuracy: 0.5734 - loss: 0.6768 - val_accuracy: 0.5135 - val_loss: 0.6744
Epoch 3/3
896/896 ━━━━━━━━━━━━━━━━━━━━ 1980s 2s/step - accuracy: 0.5773 - loss: 0.6763 - val_accuracy: 0.4795 - val_loss: 0.7042


In [24]:
# Збереження історії змін в JSON
with open('history_3_1_fin.json', 'w') as json_file:
    json.dump(history_3_1.history, json_file)

In [25]:
model_3_1.save("model_3_1_fin.h5")

In [56]:
model_3_1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 128)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer (BertLayer)    │ (None, 128, 768)       │              0 │ input_ids[0][0],       │
│                           │                        │                │ attention_mask[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 768)            │              0 │ bert_layer[0][0]       │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ binary_output (Dense)     │ (None, 1)              │            129 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 295,685 (1.13 MB)

 Trainable params: 98,561 (385.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 197,124 (770.02 KB)

In [26]:
# Отримання прогнозів для першої моделі
binary_predictions = model_3_1.predict(
    {'input_ids': test_input_ids, 'attention_mask': test_attention_mask},
    batch_size=64
)

binary_predictions = (binary_predictions > 0.5).astype(int)  # Перетворення в 0 або 1

# Розподіл прогнозів бінарної моделі
unique, counts = np.unique(binary_predictions, return_counts=True)
binary_distribution = dict(zip(unique, counts))

print("Розподіл міток:")
print(f"Нетоксичні (1): {binary_distribution.get(1, 0)}")
print(f"Токсичні (0): {binary_distribution.get(0, 0)}")

50/50 ━━━━━━━━━━━━━━━━━━━━ 39s 687ms/step
Розподіл міток:
Нетоксичні (1): 1442
Токсичні (0): 1750


In [27]:
# Оцінка моделі
print("\nКласифікаційний звіт для бінарної моделі:\n")
print(classification_report(test_binary_labels, binary_predictions, target_names=[
    "non_toxic", "toxic"
]))


Класифікаційний звіт для бінарної моделі:

              precision    recall  f1-score   support

   non_toxic       0.13      0.72      0.22       313
       toxic       0.94      0.47      0.63      2879

    accuracy                           0.49      3192
   macro avg       0.53      0.59      0.42      3192
weighted avg       0.86      0.49      0.59      3192



**Багатоміткова модель**

У якості основи для багатоміткової моделі буде взята модель №2 із даного проекту - https://github.com/T-Dzv/toxic_finder/blob/dzv-model-4/model-2.ipynb

Ця модель хоч продемонструвала схильність до визначення більшості коментарів, як токсичних. Проте попри свої недоліки вона дійсно намагалась прогнозувати всі класи токсичності, навіть якщо її результати поки далекі від ідеальних. 

Навіть при поточних результатах модель дає не нульовий recall для всіх класів, у тому числі рідких, щого не вдалось добитись у інших спробах (перша частина модулю)

Використання цієї моделі в пайплайні разом із бінарною моделлю та із донавчанням (фінтюнінг на вибірці лише із токсичних коментарів) має потенціал. 

In [28]:
# виділення із вибірки лише токсичних коментарів

# Видбірка токсичних коментарів
t_toxic_indices = np.any(train_labels == 1, axis=1)
v_toxic_indices = np.any(val_labels == 1, axis=1)

# Вхідні дані лише для токсичних прикладів
t_toxic_input_ids = train_input_ids[t_toxic_indices]
t_toxic_attention_mask = train_attention_mask[t_toxic_indices]
t_toxic_labels = train_labels[t_toxic_indices]

v_toxic_input_ids = val_input_ids[v_toxic_indices]
v_toxic_attention_mask = val_attention_mask[v_toxic_indices]
v_toxic_labels = val_labels[v_toxic_indices]

In [45]:
# Завантаження попердньо навченої моделі
from keras.models import load_model

model_path = '/kaggle/input/pretrained/model.h5'
model_3_2 = load_model(model_path, custom_objects={'BertLayer': BertLayer}, compile=False)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [46]:
model_3_2.compile(
    optimizer=Nadam(learning_rate=1e-4),
    loss=lambda y_true, y_pred: weighted_f1_loss(y_true, y_pred, class_weights),
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
)

**Фінтюнінг багатоміткової моделі**

In [47]:
# Розморозка останніх 4-х шарів BERT
for layer in model_3_2.layers:
    if isinstance(layer, BertLayer):  
        for bert_layer in layer.bert.bert.encoder.layer[-4:]:  
            bert_layer.trainable = True

model_3_2.compile(
    optimizer=Nadam(learning_rate=1e-4),
    loss=lambda y_true, y_pred: weighted_f1_loss(y_true, y_pred, class_weights),
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
)

In [48]:
# Навчання моделі
history_3_2 = model_3_2.fit(
    {
        'input_ids': t_toxic_input_ids,
        'attention_mask': t_toxic_attention_mask
    },
    t_toxic_labels,
    validation_data=(
        {
            'input_ids': v_toxic_input_ids,
            'attention_mask': v_toxic_attention_mask
        },
        v_toxic_labels),
    epochs=3, 
    batch_size=256
)

Epoch 1/3
51/51 ━━━━━━━━━━━━━━━━━━━━ 158s 3s/step - accuracy: 0.5929 - f1_metric: 0.5669 - loss: 0.9759 - precision: 0.4286 - recall: 0.8409 - val_accuracy: 0.2605 - val_f1_metric: 0.5651 - val_loss: 0.9770 - val_precision: 0.3965 - val_recall: 0.9819
Epoch 2/3
51/51 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step - accuracy: 0.2394 - f1_metric: 0.5659 - loss: 0.9762 - precision: 0.3981 - recall: 0.9787 - val_accuracy: 0.1554 - val_f1_metric: 0.5634 - val_loss: 0.9770 - val_precision: 0.3944 - val_recall: 0.9839
Epoch 3/3
51/51 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.1628 - f1_metric: 0.5630 - loss: 0.9760 - precision: 0.3944 - recall: 0.9838 - val_accuracy: 0.1329 - val_f1_metric: 0.5585 - val_loss: 0.9770 - val_precision: 0.3891 - val_recall: 0.9878


In [49]:
# Збереження історії змін в JSON
with open('history_3_2_fin.json', 'w') as json_file:
    json.dump(history_3_2.history, json_file)

In [50]:
model_3_2.save("model_3_2_fin.h5")

In [51]:
# Прогнози мультиміткової моделі на валідаціних даних
mul_model_predictions = model_3_2.predict(
    {'input_ids': v_toxic_input_ids, 'attention_mask': v_toxic_attention_mask},
    batch_size=64
)

# Перетворюємо прогнози на бінарні мітки
multilabel_predictions = (mul_model_predictions > 0.5).astype(int)
# Сумуємо значення для кожної мітки
toxic_label_counts = multilabel_predictions.sum(axis=0)

# Мітки токсичності
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

print("\nРозподіл міток багатоміткової моделі:")
for i, label in enumerate(labels):
    print(f"{label}: {toxic_label_counts[i]}")

51/51 ━━━━━━━━━━━━━━━━━━━━ 38s 660ms/step

Розподіл міток багатоміткової моделі:
toxic: 3244
severe_toxic: 2263
obscene: 3244
threat: 2720
insult: 3244
identity_hate: 3244


In [52]:
# Оцінка моделі
print("\nКласифікаційний звіт для другого підходу моделі:\n")
print(classification_report(v_toxic_labels, multilabel_predictions, target_names=[
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"
]))


Класифікаційний звіт для другого підходу моделі:

               precision    recall  f1-score   support

        toxic       0.94      1.00      0.97      3056
 severe_toxic       0.11      0.77      0.19       321
      obscene       0.53      1.00      0.69      1715
       threat       0.02      0.85      0.05        74
       insult       0.50      1.00      0.66      1614
identity_hate       0.09      1.00      0.17       294

    micro avg       0.39      0.99      0.56      7074
    macro avg       0.37      0.94      0.45      7074
 weighted avg       0.66      0.99      0.75      7074
  samples avg       0.39      0.99      0.53      7074



Після фінтюнингу на лише токсичних коментарях модель стала схольною призначати мітку 1 майже всім класам. Спробуємо альтернативно виконати фінтюнинг передавши на навчання повну вибірку даних, включаючи нетоксичні коментарі. 

In [53]:
# Завантаження попердньо навченої моделі
from keras.models import load_model

model_path = '/kaggle/input/pretrained/model.h5'
model_3_3 = load_model(model_path, custom_objects={'BertLayer': BertLayer}, compile=False)

model_3_3.compile(
    optimizer=Nadam(learning_rate=1e-4),
    loss=lambda y_true, y_pred: weighted_f1_loss(y_true, y_pred, class_weights),
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [54]:
# Розморозка останніх 4-х шарів BERT
for layer in model_3_3.layers:
    if isinstance(layer, BertLayer):  
        for bert_layer in layer.bert.bert.encoder.layer[-4:]:  
            bert_layer.trainable = True

model_3_3.compile(
    optimizer=Nadam(learning_rate=1e-5),
    loss=lambda y_true, y_pred: weighted_f1_loss(y_true, y_pred, class_weights),
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
)

In [55]:
# Навчання моделі
history_3_3 = model_3_3.fit(
    {
        'input_ids': train_input_ids,
        'attention_mask': train_attention_mask
    },
    train_labels,
    validation_data=(
        {
            'input_ids': v_toxic_input_ids,
            'attention_mask': v_toxic_attention_mask
        },
        v_toxic_labels),
    epochs=3, 
    batch_size=256
)

Epoch 1/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 1031s 2s/step - accuracy: 0.7662 - f1_metric: 0.0964 - loss: 0.9967 - precision: 0.0519 - recall: 0.7188 - val_accuracy: 0.6939 - val_f1_metric: 0.5393 - val_loss: 0.9775 - val_precision: 0.4420 - val_recall: 0.6930
Epoch 2/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 989s 2s/step - accuracy: 0.6958 - f1_metric: 0.0947 - loss: 0.9967 - precision: 0.0510 - recall: 0.6899 - val_accuracy: 0.6893 - val_f1_metric: 0.5372 - val_loss: 0.9775 - val_precision: 0.4433 - val_recall: 0.6835
Epoch 3/3
499/499 ━━━━━━━━━━━━━━━━━━━━ 989s 2s/step - accuracy: 0.6646 - f1_metric: 0.0939 - loss: 0.9968 - precision: 0.0506 - recall: 0.6685 - val_accuracy: 0.5968 - val_f1_metric: 0.5254 - val_loss: 0.9775 - val_precision: 0.4310 - val_recall: 0.6746


In [58]:
# Збереження історії змін в JSON
with open('history_3_3_fin.json', 'w') as json_file:
    json.dump(history_3_3.history, json_file)

In [59]:
model_3_3.save("model_3_3_fin.h5")

In [61]:
# Прогнози мультиміткової моделі на валідаціних даних
mul_model_predictions = model_3_3.predict(
    {'input_ids': v_toxic_input_ids, 'attention_mask': v_toxic_attention_mask},
    batch_size=64
)

# Перетворюємо прогнози на бінарні мітки
multilabel_predictions = (mul_model_predictions > 0.5).astype(int)
# Сумуємо значення для кожної мітки
toxic_label_counts = multilabel_predictions.sum(axis=0)

# Мітки токсичності
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

print("\nРозподіл міток багатоміткової моделі:")
for i, label in enumerate(labels):
    print(f"{label}: {toxic_label_counts[i]}")

51/51 ━━━━━━━━━━━━━━━━━━━━ 37s 623ms/step

Розподіл міток багатоміткової моделі:
toxic: 2462
severe_toxic: 1461
obscene: 1979
threat: 1863
insult: 1981
identity_hate: 1326


In [62]:
# Оцінка моделі
print("\nКласифікаційний звіт для другого підходу моделі:\n")
print(classification_report(v_toxic_labels, multilabel_predictions, target_names=[
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"
]))


Класифікаційний звіт для другого підходу моделі:

               precision    recall  f1-score   support

        toxic       0.95      0.77      0.85      3056
 severe_toxic       0.12      0.54      0.19       321
      obscene       0.54      0.63      0.58      1715
       threat       0.02      0.53      0.04        74
       insult       0.51      0.63      0.57      1614
identity_hate       0.09      0.43      0.15       294

    micro avg       0.43      0.67      0.53      7074
    macro avg       0.37      0.59      0.40      7074
 weighted avg       0.67      0.67      0.65      7074
  samples avg       0.40      0.68      0.44      7074



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


У другій версії моделі також прослідковується надмірне віднесення прикладів до класів токсичності (модель схильна ставити мітки 1 для більшості класів). Проте у цьому випадку принаймні модель не ставить мітки классу взагалі всім прикладам і у пайплайні з бінарною моделлю має видавати більш релеватні результати. 

**Побудова загального пайплайну прогнозів**

In [63]:
# Додання класу нетоксичних коментарів до міток тестової вибірки (всі нулі)
all_zeros_class = np.all(test_labels == 0, axis=1).astype(int)  
y_test_expanded = np.hstack((test_labels, all_zeros_class.reshape(-1, 1)))  

In [65]:
# Отримання прогнозів для першої моделі
binary_predictions = model_3_1.predict(
    {'input_ids': test_input_ids, 'attention_mask': test_attention_mask},
    batch_size=64
)
binary_predictions = (binary_predictions > 0.5).astype(int)  # Перетворення в 0 або 1

final_predictions = []
# Проходимо по кожному прикладу даних
for i in range(len(test_input_ids)):
    binary_prediction = binary_predictions[i].item()  # Прогноз бінарної моделі для поточного приклада

    if binary_prediction == 1:
        # Якщо коментар не токсичний, формуємо фінальний вектор
        final_predictions.append([0, 0, 0, 0, 0, 0, 1])  # Всі нулі + 1 на останьому індексі
    else:
        # Якщо коментар токсичний, формуємо прогноз мультимітковою моделлю
        toxic_input_ids = test_input_ids[i].reshape(1, -1)  # Приклад в форматі (1, 128)
        toxic_attention_mask = test_attention_mask[i].reshape(1, -1)

        # Прогноз мультимітковою моделлю
        multilabel_prediction = model_3_3.predict(
            {'input_ids': toxic_input_ids, 'attention_mask': toxic_attention_mask},
            batch_size=1,
            verbose=0 
        )

        # Перетворення прогнозів
        multilabel_result = (multilabel_prediction > 0.5).astype(int).flatten().tolist()
        multilabel_result.append(0)  # Дадаємо 0 в останній індекс 

        # Додаємо результат у фінальні прогнози
        final_predictions.append(multilabel_result)

# Перетворення фінальних прогнозів в numpy-масив
final_predictions = np.array(final_predictions)

50/50 ━━━━━━━━━━━━━━━━━━━━ 26s 524ms/step


In [66]:
# Оцінка моделі
print("\nКласифікаційний звіт для другого підходу моделі:\n")
print(classification_report(y_test_expanded, final_predictions, target_names=[
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate", "non_toxic"
]))

# Побудова багатоміткової матриці помилок
conf_matrices = multilabel_confusion_matrix(y_test_expanded, final_predictions)

# Приклад виводу (наприклад для "toxic")
print("Confusion matrix for 'toxic':")
print(conf_matrices[0])


Класифікаційний звіт для другого підходу моделі:

               precision    recall  f1-score   support

        toxic       0.13      0.69      0.22       304
 severe_toxic       0.02      0.51      0.03        35
      obscene       0.07      0.61      0.13       173
       threat       0.00      0.50      0.00         4
       insult       0.07      0.62      0.13       168
identity_hate       0.01      0.32      0.02        28
    non_toxic       0.94      0.47      0.63      2879

    micro avg       0.19      0.50      0.28      3591
    macro avg       0.18      0.53      0.17      3591
 weighted avg       0.77      0.50      0.53      3591
  samples avg       0.46      0.49      0.46      3591

Confusion matrix for 'toxic':
[[1444 1444]
 [  93  211]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
